In [513]:
from random import random
import numpy as np
from itertools import combinations
import math
import random

In [514]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [515]:
# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [516]:
### create a graph with planted clique of size = clique_size

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clique_size: clique size

def generate_graph(num_vertices, pr, clique_size):
    def plant_clique():
        for i in range(clique_size):
            for j in range(i + 1, clique_size):
                for l in range(j + 1, clique_size):
                    a = clique_vertices[i]
                    # print(a)
                    vec[a] += 1
                    b = clique_vertices[j]
                    # print(b)
                    vec[b] += 1
                    c = clique_vertices[l]
                    # print(c)
                    vec[c] += 1
                    add_all(a, b, c, g)

    g = np.array([np.array([np.array([0 for i in range(0, num_vertices)]) for i in range(num_vertices)]) for i in
                  range(num_vertices)])
    vec = np.array([0 for i in range(0, num_vertices)])

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    # clique_vertices = random.sample(range(num_vertices), clique_size) # is is wrong??; would violate the clique size constraint
    
    clique_vertices = np.random.choice(range(num_vertices), clique_size, replace=False)

    plant_clique()

    return g, vec, clique_vertices


In [517]:
# lst = np.random.choice(range(10), 5, replace=False)
# lst

In [518]:
# calculate the total number of edges on a graph with num_vertices vertices
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [519]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    # active_count: number of vertices that are associated with at least 1 edge
    active_count = np.count_nonzero(vec)
    # number of edges
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == 6 * math.comb(active_count, 3):
        return True
    return False

In [520]:
# remove edge
def remove_edges(g, num_vertices, vec, curr):
    vec[curr] = 0
    graph_copy = g.copy()
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if graph_copy[curr, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr, j, k, graph_copy)
    return graph_copy, vec

### Generate a graph

In [521]:
# driver program

n = 10 # number of vertices
p = 0.5 # probablity of edge being included
k = 5 # clique size

# generate graph
res = generate_graph(n, p, k)

# G graph with planted clique
# V: vector storing the number of edges associated with each vertice in the graph
# clique_vertices: the set of clique vertices
G, V, clique_vertices = res[0],  res[1], res[-1]

# print(f"graph: {G}\n")
print(f"edge-occurence vector: {V}\n")
print(f"clique vertices: {clique_vertices}\n")



edge-occurence vector: [17 20 21 16 22 27 24 23 26 17]

clique vertices: [5 6 2 8 7]



### Removal Phase

In [522]:
clique = False
iter = 0
removed = []
while not clique:
    
    iter += 1

    curr = -1
    idx_sorted = np.argsort(V)
    
    print(f"i = {iter}\nindex sorted: {idx_sorted}")
    print(f"number of edges associated with each vertex: {V}")
    # print(f'current set of clique vertices: {clique_vertices}')

    for idx in range(n):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break
    print(f"vertex removed: {curr} number of edges: {V[curr]}\n")
    A = remove_edges(G, n, V, curr)
    # print(A[1])
    G = A[0]

    clique = is_clique(G, V, n)

i = 1
index sorted: [3 0 9 1 2 4 7 6 8 5]
number of edges associated with each vertex: [17 20 21 16 22 27 24 23 26 17]
vertex removed: 3 number of edges: 16

i = 2
index sorted: [3 9 0 1 2 4 6 7 8 5]
number of edges associated with each vertex: [14 17 18  0 18 23 20 21 22 12]
vertex removed: 9 number of edges: 12

i = 3
index sorted: [3 9 0 1 4 2 7 6 8 5]
number of edges associated with each vertex: [10 12 17  0 14 20 19 18 19  0]
vertex removed: 0 number of edges: 10

i = 4
index sorted: [0 3 9 1 4 7 2 8 6 5]
number of edges associated with each vertex: [ 0 10 15  0 11 18 16 14 15  0]
vertex removed: 1 number of edges: 10

i = 5
index sorted: [0 1 3 9 4 7 2 8 6 5]
number of edges associated with each vertex: [ 0  0 12  0  7 14 13 11 12  0]
vertex removed: 4 number of edges: 7

i = 6
index sorted: [0 1 3 4 9 7 2 5 6 8]
number of edges associated with each vertex: [ 0  0  9  0  0 10 10  8 11  0]
vertex removed: 7 number of edges: 8

i = 7
index sorted: [0 1 3 4 7 9 2 5 6 8]
number of ed

In [523]:
removed

[3, 9, 0, 1, 4, 7, 2, 5]

In [524]:
clique_v = res[-1]
clique_v

array([5, 6, 2, 8, 7])

### Inclusion Phase

In [525]:
clique_vertices

array([5, 6, 2, 8, 7])

In [526]:
full_vertices = np.arange(n)
full_vertices

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [527]:
set_diff = np.setdiff1d(full_vertices, clique_vertices)
set_diff

array([0, 1, 3, 4, 9])

In [528]:
def inclusion_phase(set_diff, clique_vertices, G):
    for idx in set_diff:
        for j in range(len(clique_vertices)):
            for k in range(j+1, len(clique_vertices)):
                if G[idx, clique_vertices[j], clique_vertices[k]] != 1:
                    print(f" vertex {idx} is not connected to all clique vertices")    
                    if idx == set_diff[-1]:
                        print("reached the last element")
                        return clique_vertices
                    break
    print(f"add {idx} to clique!")
    clique_vertices = np.append(clique_vertices, idx)
    return clique_vertices
    

In [529]:
res = inclusion_phase(set_diff, clique_vertices, G)
res

 vertex 0 is not connected to all clique vertices
 vertex 0 is not connected to all clique vertices
 vertex 0 is not connected to all clique vertices
 vertex 0 is not connected to all clique vertices
 vertex 1 is not connected to all clique vertices
 vertex 1 is not connected to all clique vertices
 vertex 1 is not connected to all clique vertices
 vertex 1 is not connected to all clique vertices
 vertex 3 is not connected to all clique vertices
 vertex 3 is not connected to all clique vertices
 vertex 3 is not connected to all clique vertices
 vertex 3 is not connected to all clique vertices
 vertex 4 is not connected to all clique vertices
 vertex 4 is not connected to all clique vertices
 vertex 4 is not connected to all clique vertices
 vertex 4 is not connected to all clique vertices
 vertex 9 is not connected to all clique vertices
reached the last element


array([5, 6, 2, 8, 7])

In [530]:
clique_vertices

array([5, 6, 2, 8, 7])